In [1]:
%pip install evaluate sentence-transformers
%pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ea8a5df57828be5aa72fdd82f49c8b412109694c0e08b3192c0b274a1a1baf41
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from tqdm.auto import tqdm
import evaluate
import numpy as np
import os
import json
from sentence_transformers import SentenceTransformer, util

random_seed = 42

kaggle = True

model_name = "google/mt5-base"
model_short_name = "mt5-base"

dataset_name = ("xquad", "xquad.ro")
dataset_short_name = "xquad"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

ds = load_dataset(dataset_name[0], dataset_name[1])

ds = ds['validation'].train_test_split(test_size=0.2)
ds_train = ds['train']
ds_test = ds['test']

ds_test = ds_test.train_test_split(test_size=0.5)
ds_val = ds_test['train']
ds_test = ds_test['test']

print(len(ds_train), len(ds_val), len(ds_test))


def standardize(text: str):
    return text
    # return text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")


def process_and_tokenize(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]
    answers = dataset["answers"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    offset_mapping = inputs["offset_mapping"]

    start_pos = []
    end_pos = []

    for i, ofs in enumerate(offset_mapping):
        # Get the sample
        sample_i = sample_mapping[i]

        # Get the answer for the sample
        answer = answers[sample_i]

        # Get the start and end character of the answer
        start = answer["answer_start"][0]
        end = answer["answer_start"][0] + len(answer["text"][0])

        # Get the sequence ids
        seq_ids = inputs.sequence_ids(i)

        # Get the start and end token positions
        start_context = seq_ids.index(1)
        end_context = next(j - 1 for j in range(start_context, len(seq_ids)) if seq_ids[j] != 1)

        if ofs[start_context][0] > start or ofs[end_context][1] < end:  # If it's impossible
            start_pos.append(0)
            end_pos.append(0)
        else:  # Get and append start and end position
            start_pos.append(
                next((j - 1 for j in range(start_context, end_context + 1) if ofs[j][0] > start), end_context))
            end_pos.append(
                next((j + 1 for j in range(end_context, start_context - 1, -1) if ofs[j][1] < end), start_context))

            # if i < len(dataset['id']) and dataset['id'][i] == '56d6f3500d65d21400198292':
            #     print(start_context, end_context, end_pos[-1], end_pos[-1])

    inputs["start_positions"] = start_pos
    inputs["end_positions"] = end_pos

    inputs.pop("overflow_to_sample_mapping")
    inputs.pop("offset_mapping")

    return inputs


ds_tok_train = ds_train.map(lambda x: process_and_tokenize(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64,
                            remove_columns=ds_train.column_names)

print(len(ds_tok_train))


def process_and_tokenize_val_test(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_i = sample_mapping[i]
        example_ids.append(dataset["id"][sample_i])

        seq_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        for j, ofs in enumerate(offset):
            inputs['offset_mapping'][i][j] = ofs if seq_ids[j] == 1 else None

    inputs["example_id"] = example_ids

    inputs.pop("overflow_to_sample_mapping")

    return inputs


ds_tok_val = ds_val.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True,
                        batch_size=64, remove_columns=ds_val.column_names)
ds_tok_test = ds_test.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True,
                          batch_size=64, remove_columns=ds_test.column_names)

print(len(ds_tok_val), len(ds_tok_test))

squad_metric = evaluate.load("squad")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")


def compute_metrics(start_logits: list, end_logits: list, tok: dict, dataset: Dataset, max_answer_length: int = 30,
                    eval_answer: SentenceTransformer = SentenceTransformer(
                        "BlackKakapo/stsb-xlm-r-multilingual-ro")) -> dict:
    extracted_features = {}

    # Group extracted features by ids
    for i, feature in enumerate(tok):
        if feature["example_id"] not in extracted_features:
            extracted_features[feature["example_id"]] = [i]
        else:
            extracted_features[feature["example_id"]].append(i)

    golds_squad = [{"id": data["id"], "answers": data["answers"]} for data in dataset]
    preds_squad = []

    golds_bleu = []
    preds_bleu = []

    for data in tqdm(dataset):
        answers = []

        # Iterate over all the extracted features
        for i in extracted_features[data["id"]]:
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            offs = tok[i]["offset_mapping"]

            # Get all combinations of start and end positions
            for start_i in range(len(start_logit)):
                for end_i in range(len(end_logit)):
                    # Continue on wrong answers
                    if offs[start_i] is None \
                            or offs[end_i] is None \
                            or end_i < start_i \
                            or end_i - start_i + 1 > max_answer_length:
                        continue

                    # Add text and score
                    answer = {
                        "answer": data["context"][offs[start_i][0]: offs[end_i][1]],
                        "score": start_logit[start_i] + end_logit[end_i]
                    }
                    answers.append(answer)

        preds_squad.append(
            {"id": data['id'], "prediction_text": max(answers, key=lambda x: x["score"])['answer']}
            if len(answers) > 0 else
            {"id": data['id'], "prediction_text": ""}
        )

        preds_bleu.append(
            max(answers, key=lambda x: x["score"])['answer']
            if len(answers) > 0 else
            ""
        )

        max_i = 0
        max_len = len(data["answers"]["text"][max_i])
        for i in range(1, len(data["answers"]['text'])):
            if len(data["answers"]["text"][i]) > max_len:
                max_len = len(data["answers"]["text"][i])
                max_i = i

        golds_bleu.append(data["answers"]['text'][max_i])

    return {
        "squad": squad_metric.compute(predictions=preds_squad, references=golds_squad),
        "bleu": bleu_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "rouge": rouge_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "semantic_similarity": util.pytorch_cos_sim(eval_answer.encode(preds_bleu, convert_to_tensor=True),
                                                    eval_answer.encode(golds_bleu,
                                                                       convert_to_tensor=True)).mean().item() * 100
    }


def output_metrics_to_file(metrics: dict, metric_type: str = None, lr: float = None, epoch: int = None):
    filename = os.path.join("/kaggle/working/" if kaggle else ".", "results",
                            f"{model_short_name}-{dataset_short_name}-type_{metric_type}-lr_{lr}-epoch_{epoch:02}.json" if lr is not None and epoch is not None and metric_type is not None else f"{model_short_name}-{dataset_short_name}.json")
    with open(filename, "w") as f:
        json.dump(metrics, f, indent=4)


def save_model(trainer: Trainer, lr: float, epoch: int):
    trainer.save_model(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models",
                                    f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))


def load_model(lr: float, epoch: int) -> AutoModelForQuestionAnswering:
    return AutoModelForQuestionAnswering.from_pretrained(
        os.path.join("/kaggle/working/" if kaggle else ".", "results", "models",
                     f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))


all_models_eval = None


def tet_loop(lr_list: list, epochs: int, batch_size: int) -> None:
    global all_models_eval
    all_models_eval = {"validation": {lr: {epoch: [] for epoch in range(1, epochs + 1)} for lr in lr_list},
                       "test": {lr: {epoch: [] for epoch in range(1, epochs + 1)} for lr in lr_list}}

    eval_answer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")

    for lr in lr_list:
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)

        warmup_steps = int(len(ds_tok_train) / batch_size * 1 / 10)
        args = TrainingArguments(
            output_dir="./results",
            eval_strategy="no",
            save_strategy="epoch",
            learning_rate=lr,
            num_train_epochs=1,
            weight_decay=0.01,
            per_device_train_batch_size=batch_size,
            report_to="none",
            save_total_limit=1,
            warmup_steps=warmup_steps
        )
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=ds_tok_train,
            eval_dataset=ds_tok_val,
            tokenizer=tokenizer
        )

        for epoch in range(1, epochs + 1):
            print(f"Training with lr={lr} and epoch={epoch}")

            trainer.train()

            preds = trainer.predict(ds_tok_val)
            start_logits, end_logits = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits, end_logits, ds_tok_val, ds_val, 30, eval_answer)

            # output_metrics_to_file(computed_metrics, metric_type='validation', lr=lr, epoch=epoch)
            print(f"Validation for lr {lr} epoch {epoch}: ")
            # print(computed_metrics)
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])

            all_models_eval['validation'][lr][epoch] = computed_metrics

            preds = trainer.predict(ds_tok_test)
            start_logits2, end_logits2 = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits2, end_logits2, ds_tok_test, ds_test, 30, eval_answer)

            output_metrics_to_file(computed_metrics, metric_type='test', lr=lr, epoch=epoch)
            all_models_eval['test'][lr][epoch] = computed_metrics

            print(f"Test for lr {lr} epoch {epoch}: ")
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])

            # save_model(trainer, lr, epoch)

        try:
            del trainer
            del model
            # device = cuda.get_current_device()
            # device.reset()
        except Exception as e:
            print(e)

            print(f"Finished training, validating and testing with lr={lr} and epochs={epochs}")

tet_loop(lr_list=[1e-4, 1e-3, 1e-5], epochs=16, batch_size=4)
output_metrics_to_file(all_models_eval)
!zip results_ml.zip results/*.json

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

Validation for lr 1e-05 epoch 16: 
{'exact_match': 0.0, 'f1': 4.759111178872632}
16.406571865081787


  0%|          | 0/119 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Test for lr 1e-05 epoch 16: 
{'exact_match': 0.0, 'f1': 6.578310560020893}
15.61136245727539
  adding: results/mt5-base-xquad.json (deflated 85%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_01.json (deflated 53%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_02.json (deflated 52%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_03.json (deflated 53%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_04.json (deflated 53%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_05.json (deflated 52%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_06.json (deflated 52%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_07.json (deflated 53%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_08.json (deflated 54%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_09.json (deflated 53%)
  adding: results/mt5-base-xquad-type_test-lr_0.0001-epoch_10.json (deflated 52%)
  adding: results/mt5-base-xquad-t